<a href="https://colab.research.google.com/github/iraidaantropova/ApacheSpark/blob/main/HW3_PySpark_SinitsaI_DF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285398 sha256=b2085486e6aecdf588a9098b574db4e8dd0946aa4db23b968e0eed483c6c34d8
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [2]:
#точка входа в spark

from pyspark.sql import SparkSession
import pyspark.sql.functions as F


spark = SparkSession.builder\
        .master('local[4]')\
        .appName('Lesson_2')\
        .config('spark.ui.port', '4050')\
        .config('spark.executor.instances', 2)\
        .config('spark.executor.memory', '5g')\
        .config('spark.executor.cores', 2)\
        .getOrCreate()

sc = spark.sparkContext

In [3]:
spark

In [4]:
#считывание данных
# Прочитайте содержимое каждого файла в датафрейм и выведите первые несколько строк
# Применив cast к колонке можно изменить тип данных (F.col().cast(IntegerType()).alias())
# Приведите все данные к правильному типу, либо считайе сразу верно

train = spark.read.csv('train.csv', header=True)
test = spark.read.csv('test.csv', header=True)

In [5]:
train.show()

+-------------+----+-----------+--------+---+------+----------+-----+---------+-------+---+---------+--------+----+----+----+---------+-------+------------+----+-----------+
|battery_power|blue|clock_speed|dual_sim| fc|four_g|int_memory|m_dep|mobile_wt|n_cores| pc|px_height|px_width| ram|sc_h|sc_w|talk_time|three_g|touch_screen|wifi|price_range|
+-------------+----+-----------+--------+---+------+----------+-----+---------+-------+---+---------+--------+----+----+----+---------+-------+------------+----+-----------+
|          842|   0|        2.2|       0|  1|     0|         7|  0.6|      188|      2|  2|       20|     756|2549|   9|   7|       19|      0|           0|   1|          1|
|         1021|   1|        0.5|       1|  0|     1|        53|  0.7|      136|      3|  6|      905|    1988|2631|  17|   3|        7|      1|           1|   0|          2|
|          563|   1|        0.5|       1|  2|     1|        41|  0.9|      145|      5|  6|     1263|    1716|2603|  11|   2|     

In [6]:
#объединение train и test

data = train.drop('price_range').unionByName(test.drop('id'))

In [7]:
data.count()

3000

In [8]:
import pandas as pd

In [9]:
train = train.drop('price_range')
test = test.drop('id')

pandas_train = train.toPandas()
pandas_test = test.toPandas()

merged_pandas_df = pd.concat([pandas_train, pandas_test], axis=0)

spark_df = spark.createDataFrame(merged_pandas_df)

In [10]:
spark_df.printSchema()

root
 |-- battery_power: string (nullable = true)
 |-- blue: string (nullable = true)
 |-- clock_speed: string (nullable = true)
 |-- dual_sim: string (nullable = true)
 |-- fc: string (nullable = true)
 |-- four_g: string (nullable = true)
 |-- int_memory: string (nullable = true)
 |-- m_dep: string (nullable = true)
 |-- mobile_wt: string (nullable = true)
 |-- n_cores: string (nullable = true)
 |-- pc: string (nullable = true)
 |-- px_height: string (nullable = true)
 |-- px_width: string (nullable = true)
 |-- ram: string (nullable = true)
 |-- sc_h: string (nullable = true)
 |-- sc_w: string (nullable = true)
 |-- talk_time: string (nullable = true)
 |-- three_g: string (nullable = true)
 |-- touch_screen: string (nullable = true)
 |-- wifi: string (nullable = true)



In [11]:
#Рассчитайте соотношение сторон телефона и экрана
# Выведите отсортированное распределение плотности пикселей (1 балла)
# в пикселях на дюйм

data = data.withColumn('ppi',
                      (F.pow(F.pow(F.col('px_height'), F.lit(2)) + F.pow(F.col('px_width'), F.lit(2)), F.lit(0.5)) /
        F.pow(F.pow(F.col('sc_h'), F.lit(2)) + F.pow(F.col('sc_w'), F.lit(2)), F.lit(0.5)))
    )
data.show()

+-------------+----+-----------+--------+---+------+----------+-----+---------+-------+---+---------+--------+----+----+----+---------+-------+------------+----+------------------+
|battery_power|blue|clock_speed|dual_sim| fc|four_g|int_memory|m_dep|mobile_wt|n_cores| pc|px_height|px_width| ram|sc_h|sc_w|talk_time|three_g|touch_screen|wifi|               ppi|
+-------------+----+-----------+--------+---+------+----------+-----+---------+-------+---+---------+--------+----+----+----+---------+-------+------------+----+------------------+
|          842|   0|        2.2|       0|  1|     0|         7|  0.6|      188|      2|  2|       20|     756|2549|   9|   7|       19|      0|           0|   1|  66.3287847944442|
|         1021|   1|        0.5|       1|  0|     1|        53|  0.7|      136|      3|  6|      905|    1988|2631|  17|   3|        7|      1|           1|   0|126.53313490223783|
|          563|   1|        0.5|       1|  2|     1|        41|  0.9|      145|      5|  6|    

In [12]:
# Сохраните результат в csv sep=';', encoding='cp1251'
# с колонками id, плотность пикселей и общей инфой в формате "2 ядра 2 гига"

data.select(
    F.monotonically_increasing_id(),
    'ppi',
    F.concat_ws(' ',
        F.col('n_cores'),
        F.lit('ядра')).alias('desc'))\
.write.option('encoding', 'cp1252').csv('results.csv', mode='overwrite', sep=';')